In [1]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Test').getOrCreate()
spark

In [3]:
df_pyspark=spark.read.parquet('cleaned_postings.parquet')
df_pyspark=spark.read.option('header','true').parquet('cleaned_postings.parquet',inferSchema=True)


In [4]:
type(df_pyspark)
df_pyspark.printSchema()

root
 |-- job_id: long (nullable = true)
 |-- company_name: string (nullable = true)
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- skills_desc: string (nullable = true)
 |-- normalized_salary: double (nullable = true)
 |-- formatted_experience_level: string (nullable = true)
 |-- formatted_work_type: string (nullable = true)
 |-- remote_allowed: boolean (nullable = true)
 |-- posting_domain: string (nullable = true)
 |-- location: string (nullable = true)
 |-- listed_time: double (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- extracted_skills: string (nullable = true)



In [5]:
from pyspark.sql import functions as F
df_rows = df_pyspark.withColumn("extracted_individual_skill", F.explode(F.split(F.col("extracted_skills"), "\|")))
df_rows.show(50)

<>:2: SyntaxWarning: invalid escape sequence '\|'
<>:2: SyntaxWarning: invalid escape sequence '\|'
C:\Users\harsh\AppData\Local\Temp\ipykernel_20624\2931386557.py:2: SyntaxWarning: invalid escape sequence '\|'
  df_rows = df_pyspark.withColumn("extracted_individual_skill", F.explode(F.split(F.col("extracted_skills"), "\|")))


+---------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------------+-------------------+--------------+--------------+--------------------+-----------+--------+--------------------+--------------------------+
|   job_id|        company_name|               title|         description|         skills_desc|normalized_salary|formatted_experience_level|formatted_work_type|remote_allowed|posting_domain|            location|listed_time|zip_code|    extracted_skills|extracted_individual_skill|
+---------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------------+-------------------+--------------+--------------+--------------------+-----------+--------+--------------------+--------------------------+
|   921716|Corcoran Sawyer S...|Marketing Coordin...|Job descriptionA ...|Requirements: \n\...|          38480.0|                      NULL|          Full-ti

In [6]:
(
    df_rows
    .filter(F.col("extracted_individual_skill").isNotNull())
    .filter(F.trim(F.col("extracted_individual_skill")) != "")
    .groupBy("extracted_individual_skill")
    .count()
    .orderBy(F.desc("count"))
    .show(50, truncate=False)
)


+--------------------------+-----+
|extracted_individual_skill|count|
+--------------------------+-----+
|excel                     |17851|
|go                        |8762 |
|agile                     |5932 |
|c                         |5406 |
|sql                       |5039 |
|python                    |4536 |
|aws                       |3034 |
|forecasting               |2956 |
|azure                     |2825 |
|rest                      |2712 |
|java                      |2631 |
|oracle                    |2301 |
|statistics                |1943 |
|javascript                |1904 |
|express                   |1791 |
|machine learning          |1704 |
|scrum                     |1527 |
|jira                      |1480 |
|r                         |1479 |
|react                     |1358 |
|ci/cd                     |1325 |
|html                      |1237 |
|c++                       |1225 |
|tableau                   |1214 |
|spring                    |1143 |
|git                

In [7]:
skill_map = {
    "node.js": "nodejs",
    "ci / cd": "ci/cd",
    "amazon web services": "aws",
    "natural language processing": "nlp",
    "golang": "go",
    "postgres": "postgresql",
    "c #": "c#",
    "apache spark": "spark",
    "ab testing": "a/b testing"
    #add more if needed
}
mapping_expr = F.create_map(
    *[F.lit(x) for pair in skill_map.items() for x in pair]
)

df_norm = df_rows.withColumn(
    "skill",
    F.coalesce(mapping_expr[F.col("extracted_individual_skill")], F.col("extracted_individual_skill"))
)

In [14]:
(
    df_norm
    .filter(F.col("skill").isNotNull())
    .filter(F.trim(F.col("skill")) != "")
    .dropDuplicates(["job_id", "skill"])
    .groupBy("skill")
    .count()
    .orderBy(F.desc("count"))
    .show(50, truncate=False)
)

+-----------------------+-----+
|skill                  |count|
+-----------------------+-----+
|excel                  |17851|
|go                     |8893 |
|agile                  |5932 |
|c                      |5406 |
|sql                    |5039 |
|python                 |4536 |
|aws                    |3118 |
|forecasting            |2956 |
|azure                  |2825 |
|rest                   |2712 |
|java                   |2631 |
|oracle                 |2301 |
|statistics             |1943 |
|javascript             |1904 |
|express                |1791 |
|machine learning       |1704 |
|scrum                  |1527 |
|jira                   |1480 |
|r                      |1479 |
|react                  |1358 |
|ci/cd                  |1339 |
|html                   |1237 |
|c++                    |1225 |
|tableau                |1214 |
|spring                 |1143 |
|git                    |1134 |
|kubernetes             |1091 |
|c#                     |1077 |
|docker 

In [9]:
skills_by_level = (
    df_norm
    .filter(F.col("skill").isNotNull())
    .filter(F.trim(F.col("skill")) != "")
    .fillna({"formatted_experience_level": "Unknown"})
    .groupBy("formatted_experience_level", "skill")
    .agg(F.countDistinct("job_id").alias("job_count"))
    .orderBy("formatted_experience_level", F.desc("job_count"))
)

skills_by_level.show(50, truncate=False)

+--------------------------+-----------------------+---------+
|formatted_experience_level|skill                  |job_count|
+--------------------------+-----------------------+---------+
|Associate                 |excel                  |1961     |
|Associate                 |go                     |614      |
|Associate                 |agile                  |429      |
|Associate                 |sql                    |351      |
|Associate                 |c                      |303      |
|Associate                 |python                 |267      |
|Associate                 |forecasting            |244      |
|Associate                 |statistics             |181      |
|Associate                 |oracle                 |158      |
|Associate                 |azure                  |148      |
|Associate                 |aws                    |127      |
|Associate                 |r                      |124      |
|Associate                 |jira                   |122

In [ ]:
from pyspark.sql.functions import expr
job_skills = (
    df_norm
    .filter(F.col("skill").isNotNull())
    .filter(F.trim(F.col("skill")) != "")
    .groupBy("job_id")
    .agg(F.collect_set("skill").alias("skills"))
)
# job_skills.show(50, truncate=False)
skill_pairs = (
    job_skills
    .withColumn(
        "skill_pairs",
        expr("""
            transform(
                filter(
                    sequence(0, size(skills) - 1),
                    i -> i < size(skills) - 1
                ),
                i -> transform(
                    sequence(i + 1, size(skills) - 1),
                    j -> struct(
                        skills[i] as skill_a,
                        skills[j] as skill_b
                    )
                )
            )
        """)
    )
    .select("job_id", F.explode("skill_pairs").alias("pairs"))
    .select("job_id", F.explode("pairs").alias("pair"))
    .select("job_id","pair.skill_a", "pair.skill_b")
)
skill_bundle_counts = (
    skill_pairs
    .groupBy("skill_a", "skill_b")
    .agg(F.countDistinct("job_id").alias("job_count"))
    .orderBy(F.desc("job_count"))
)
skill_bundle_counts.show(50, truncate=False)

+----------+-------------------------------------------------------------------------------------------------------------------------+
|job_id    |skills                                                                                                                   |
+----------+-------------------------------------------------------------------------------------------------------------------------+
|83789755  |[go]                                                                                                                     |
|115639136 |[excel]                                                                                                                  |
|134286190 |[excel]                                                                                                                  |
|136743465 |[go]                                                                                                                     |
|175485704 |[mysql, javascript, html, css, php]        

In [11]:
top_pairs = (
    skill_bundle_counts
    .filter(F.col("job_count") >= 200) 
    .orderBy(F.desc("job_count"))
)

top_pairs.show(50, truncate=False)

+----------------+----------+---------+
|skill_a         |skill_b   |job_count|
+----------------+----------+---------+
|python          |sql       |1845     |
|agile           |scrum     |1337     |
|aws             |python    |1310     |
|aws             |azure     |1296     |
|excel           |go        |1280     |
|agile           |sql       |1220     |
|forecasting     |excel     |1143     |
|c               |c#        |1069     |
|python          |java      |1046     |
|agile           |python    |1022     |
|agile           |java      |1017     |
|agile           |aws       |999      |
|azure           |sql       |997      |
|excel           |sql       |959      |
|agile           |azure     |957      |
|aws             |sql       |914      |
|sql server      |sql       |892      |
|machine learning|python    |862      |
|c               |python    |851      |
|c               |sql       |829      |
|javascript      |html      |815      |
|oracle          |sql       |811      |


In [12]:
job_salary = df_rows.select("job_id", "normalized_salary").filter(F.col("normalized_salary") > 0)

pair_salary = (
    skill_pairs
    .join(job_salary, on="job_id", how="inner")
    .groupBy("skill_a", "skill_b")
    .agg(
        F.countDistinct("job_id").alias("job_count"),
        F.avg("normalized_salary").alias("avg_salary")
    )
    .filter(F.col("job_count") >= 100)
    .orderBy(F.desc("avg_salary"))
)

pair_salary.show(50, truncate=False)


+----------------+-----------------------+---------+------------------+
|skill_a         |skill_b                |job_count|avg_salary        |
+----------------+-----------------------+---------+------------------+
|javascript      |css                    |109      |942102.3331686746 |
|html            |css                    |147      |828895.9891440502 |
|aws             |azure                  |349      |821656.2467959304 |
|javascript      |html                   |153      |616075.7445086705 |
|machine learning|artificial intelligence|211      |177466.58050161813|
|spark           |python                 |126      |163944.17513455328|
|machine learning|aws                    |115      |163172.5682123656 |
|c++             |python                 |192      |162428.76901408448|
|machine learning|python                 |309      |162363.54731315628|
|data pipelines  |python                 |100      |158983.5836637589 |
|c               |aws                    |108      |156992.44397

In [13]:
top_pairs.write.mode("overwrite").parquet("data/skill_bundles_top.parquet")
pair_salary.write.mode("overwrite").parquet("data/skill_bundles_salary.parquet")
